# Algorithms used: ***SVM, RandomForestClassifier, XGBoost, Node2vec***
## **For SVM** : got 45 - 50 % accuracy
## **For RandomForesrtClassifier** : got 80 - 85 % accuracy
## **For XGBoost** : got 92 - 95 % accuracy
## **For XGBoost + Node2vec** : got 94 - 98 % accuracy

## **Dataset Link** : https://snap.stanford.edu/data/ego-Facebook.html

# **Algorithm used : SVM**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### **Advantages of Support Vector Machines (SVM)**

#### **1. Simplicity**
- SVM is **straightforward to implement** for binary classification tasks, making it an accessible choice for many applications.
- When using the **linear kernel**, the algorithm trains efficiently, ensuring quick setup and deployment.

#### **2. Efficiency**
- SVM is particularly effective with **small to medium-sized datasets**, delivering accurate and reliable results without requiring excessive computational resources.

In [ ]:
def load_features(file_path):
    data = pd.read_csv(file_path, header=None, delim_whitespace=True).values
    return data

def load_edges(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, header=None)
    return data.values

def align_features(egofeat, feat):
    # Check if dimensions match
    if egofeat.shape[1] != feat.shape[1]:
        min_cols = min(egofeat.shape[1], feat.shape[1])
        egofeat = egofeat[:, :min_cols]
        feat = feat[:, :min_cols]
    return egofeat, feat

def prepare_data(edges_file, feat_file, egofeat_file):
    edges = load_edges(edges_file)
    feat = load_features(feat_file)
    egofeat = load_features(egofeat_file)
    egofeat, feat = align_features(egofeat, feat)
    features = np.vstack([egofeat, feat])
    labels = np.array([1 if i % 2 == 0 else 0 for i in range(features.shape[0])])
    return features, labels

### **Support Vector Machine (SVM) - Explanation**

---

#### **Objective Function**
The objective of SVM is to minimize the following function:

$$
\min_{\mathbf{w}, b} \frac{1}{2} \|\mathbf{w}\|^2
$$

where:
- $(\mathbf{w})$ is the weight vector (defining the hyperplane).
- $(b)$ is the bias term.
- $(\|\mathbf{w}\|^2)$ is the squared norm of the weight vector, which is minimized to maximize the margin.

---

#### **Constraints**
For linearly separable data, the SVM must satisfy:

$$
y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1, \quad \forall i
$$

---

#### **Soft Margin SVM (Non-Separable Data)**
For non-linearly separable data, a slack variable $(\xi_i)$ is introduced to allow some misclassifications. The optimization problem becomes:

$$
\min_{\mathbf{w}, b, \boldsymbol{\xi}} \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^{n} \xi_i
$$

subject to:

$$
y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0
$$

where:
- $(\xi_i)$ measures the misclassification for the sample.
- \(C\) is a regularization parameter that controls the trade-off between maximizing the margin and minimizing classification errors.

---

#### **Kernel SVM**
When data is not linearly separable, it is transformed into a higher-dimensional space using a kernel function. \$(K(\mathbf{x}_i, \mathbf{x}_j))$ The decision boundary is then computed as:

$$
f(\mathbf{x}) = \text{sign}\left( \sum_{i=1}^{n} \alpha_i y_i K(\mathbf{x}_i, \mathbf{x}) + b \right)
$$

---

#### **Decision Boundary**
The decision function for SVM is given by:

$$
f(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b
$$

#### **Margin**
The margin is defined as:

$$
\text{Margin} = \frac{2}{\|\mathbf{w}\|}
$$

Maximizing the margin ensures better generalization of the classifier.


In [ ]:
def main():
    edges_file = "./facebook_data/0.edges"
    feat_file = "./facebook_data/0.feat"
    egofeat_file = "./facebook_data/0.egofeat"

    # Load and prepare data
    features, labels = prepare_data(edges_file, feat_file, egofeat_file)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train SVM
    clf = SVC(kernel="linear", random_state=42)
    clf.fit(X_train, y_train)

    # Test the model
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy*100:.2f} %")

if __name__ == "__main__":
    main()

<ipython-input-14-95ae4f63b8f6>:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, header=None)
<ipython-input-14-95ae4f63b8f6>:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, header=None, delim_whitespace=True).values
<ipython-input-14-95ae4f63b8f6>:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, header=None, delim_whitespace=True).values


Accuracy: 47.14 %


### Disadvantages of the Current Implementation : **Getting very low accuracy**

1. **Dummy Labels**  
   - The labels are randomly generated (1 for even indices, 0 for odd), which doesn't reflect real-world data.  
   - This approach is a heuristic and does not utilize meaningful labels from the dataset.  
   - **Solution:** Replace this heuristic with meaningful labels, such as using the circles from the `.circles` file for classification tasks.

2. **No Use of Edges**  
   - While the edge data is loaded, it is not utilized in the model.  
   - This ignores valuable information about the graph's structure, such as relationships between nodes.  
   - **Solution:** Incorporate edge data into the model to capture relational information, such as adjacency matrices or features derived from the graph topology.

3. **Feature Engineering**  
   - The implementation does not include graph-based features like:
     - Node degree (number of connections for each node).
     - Clustering coefficient (measure of local connectivity around a node).
     - Neighbor-based features (e.g., average or sum of neighbor features).  
   - **Solution:** Perform additional feature engineering to extract graph-based features that better represent the data and improve classification performance.

# **Algorithm used : Random Forest Classifier**

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### **1. Load the Graph (`load_graph`)**
#### **Purpose:**
Constructs a graph from an edge list file.

#### **Input:**
- File containing edges, where each line represents an edge (e.g., `u v`).

#### **Output:**
- A graph \( G \) represented as a NetworkX object.

In [ ]:
def load_graph(edges_file):
    G = nx.Graph()
    with open(edges_file, 'r') as f:
        for line in f:
            u, v = map(int, line.strip().split())
            G.add_edge(u, v)
    return G

### **2. Load Features (`load_features`)**
#### **Purpose:**
Loads features for each node from the feature file (`feat_file`) and ego features (`egofeat_file`) and combines them.

#### **Input:**
- `feat_file`: Node features.
- `egofeat_file`: Ego (self) features.

#### **Output:**
- A single dataframe containing features for all nodes.

In [ ]:
def load_features(feat_file, egofeat_file):
    features = pd.read_csv(feat_file, header=None, delimiter=' ')
    egofeat = pd.read_csv(egofeat_file, header=None, delimiter=' ')
    return pd.concat([egofeat, features], ignore_index=True)

### **3. Generate Training Data (`generate_training_data`)**
#### **Purpose:**
Creates labeled data for supervised link prediction.

#### **Steps:**
1. **Edges:** Extract all edges from the graph \( G \) using `G.edges()`.
2. **Non-Edges:** Use `nx.non_edges()` to sample an equal number of non-existent edges.
3. **Labels:**  
   - Assign \( 1 \) to actual edges.  
   - Assign \( 0 \) to sampled non-edges.
4. **Feature Extraction:**  
   - For each edge or non-edge \( (u, v) \), concatenate the feature vectors of nodes \( u \) and \( v \).  
   - Combine all edge and non-edge features into \( X \), and labels into \( y \).

#### **Algorithm Used:**
- Random sampling for non-edges.
- Feature concatenation for supervised learning.

#### **Output:**
- \( X \): Feature matrix (each row corresponds to an edge/non-edge pair).
- \( y \): Labels indicating whether the pair represents an edge (\( 1 \)) or non-edge (\( 0 \)).


In [ ]:
def generate_training_data(G, features):
    edges = list(G.edges())
    non_edges = list(nx.non_edges(G))
    np.random.shuffle(non_edges)

    labels = [1] * len(edges) + [0] * len(non_edges[:len(edges)])

    def extract_features(u, v):
        return np.concatenate([features.iloc[u].values, features.iloc[v].values])

    edge_features = [extract_features(u, v) for u, v in edges]
    non_edge_features = [extract_features(u, v) for u, v in non_edges[:len(edges)]]

    X = np.array(edge_features + non_edge_features)
    y = np.array(labels)
    return X, y

### **4. Train and Evaluate Model (`train_and_evaluate`)**
#### **Purpose:**
Splits the data into training and test sets, trains a classifier, and evaluates its performance.

#### **Algorithm Used:**
- **Model:** Random Forest Classifier (ensemble method).  
  - Random Forest constructs multiple decision trees and outputs the majority vote (classification) or average (regression).

#### **Key Parameters:**
- `n_estimators=100`: The number of decision trees in the forest.

#### **Steps:**
1. **Train-Test Split:**  
   - Split the data:  
     - 80% of the data for training.  
     - 20% of the data for testing.
2. **Evaluation Metrics:**  
   - **Accuracy:** Proportion of correctly predicted samples.  
   - **Classification Report:** Includes precision, recall, F1-score, etc.

In [ ]:
def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f} %")
    print(classification_report(y_test, y_pred))
    return model

### **Advantages**

#### **1. Robustness to Overfitting**
Random Forest reduces overfitting by averaging the results of multiple decision trees.  
##### Formula for aggregation:
$[
\hat{y} = \text{Majority Vote or Mean}\left(\text{predictions from } T_1, T_2, \dots, T_n\right)
]$
- $( \hat{y} )$: Final prediction.  
- $( T_i )$: Individual decision tree in the forest.

---

#### **2. Handles High-Dimensional Data**
Random Forest can handle datasets with a large number of features, as it selects a subset of features for each split:  
##### Formula:
$[
\text{Features Per Split} =
\begin{cases}
\sqrt{d} & \text{for classification} \\
d/3 & \text{for regression}
\end{cases}
]$
- \( d \): Total number of features.

---

#### **3. Resilient to Missing and Noisy Data**
The ensemble nature ensures stability even if some data is noisy or missing.

---

#### **4. Feature Importance**
Random Forest provides a mechanism to measure feature importance:  
##### Formula:
\$[
\text{Feature Importance} = \frac{\sum (\text{Decrease in Impurity for Splits on Feature})}{\text{Total Trees}}
\$]

---

#### **5. Non-Parametric**
Random Forest does not make any assumptions about the data distribution (e.g., linearity or normality).


### **Getting better accuracy with single dataset**

In [ ]:
if __name__ == "__main__":
    # Paths to files
    edges_file = "./facebook_data/0.edges"
    feat_file = "./facebook_data/0.feat"
    egofeat_file = "./facebook_data/0.egofeat"

    # Load graph and features
    G = load_graph(edges_file)
    features = load_features(feat_file, egofeat_file)

    # Generate training data
    X, y = generate_training_data(G, features)

    # Train and evaluate the model
    model = train_and_evaluate(X, y)

Accuracy: 82.24 %
              precision    recall  f1-score   support

           0       0.80      0.85      0.82       490
           1       0.85      0.79      0.82       518

    accuracy                           0.82      1008
   macro avg       0.82      0.82      0.82      1008
weighted avg       0.82      0.82      0.82      1008



# **Algorithm used : XGBoost Classifier**

XGBoost is an ensemble learning algorithm based on **gradient boosting**.

**Objective**: Minimize a loss function using a series of decision trees.

The loss function $(L(\phi))$ is defined as:

$[
L(\phi) = \sum_{i=1}^{n} \ell(y_i, \hat{y}_i) + \sum_{k=1}^{K} \Omega(f_k)
]$

Where:
- $(\ell)$: Loss function (e.g., logistic loss for classification).
- $(\hat{y}_i)$: Predicted label for the \(i\)-th data point.
- $(\Omega(f_k))$: Regularization term for each tree \(f_k\).
- $(K)$: Total number of trees in the ensemble.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import networkx as nx
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

### **load_edges(file_path) :**
Loads the edge list from a file, assuming edges are stored in a space-separated format.
Output: A DataFrame with source and target columns, representing graph edges.

In [ ]:
def load_edges(file_path):
    edges = pd.read_csv(file_path, sep=" ", header=None, names=["source", "target"])
    return edges

### **load_features(feat_path, ego_feat_path) :**
Combines node features (feat_file) and ego node features (egofeat_file).
Output: A concatenated DataFrame of features.

In [ ]:
def load_features(feat_path, ego_feat_path):
    features = pd.read_csv(feat_path, sep=" ", header=None)
    ego_features = pd.read_csv(ego_feat_path, sep=" ", header=None)
    return pd.concat([features, ego_features], axis=0).reset_index(drop=True)

### **load_circles(circle_path) :**
Loads circle (community) labels. Each line in the file represents a circle with its associated nodes.
Output: A list of lists, where each inner list contains nodes belonging to a circle.

In [ ]:
def load_circles(circle_path):
    circles = []
    with open(circle_path, 'r') as f:
        for line in f:
            nodes = line.strip().split('\t')[1:]  # Exclude the circle name
            circles.append(nodes)
    return circles

### **prepare_dataset(files) :**
Processes multiple data files (edges, features, circles) and constructs a graph using NetworkX.

In [ ]:
def prepare_dataset(files):
    G = nx.Graph()
    features = []
    labels = []

    for file in files:
        edges_file = f"{file}.edges"
        feat_file = f"{file}.feat"
        ego_feat_file = f"{file}.egofeat"
        circle_file = f"{file}.circles"

        edges = load_edges(edges_file)
        G.add_edges_from(edges.values)

        node_features = load_features(feat_file, ego_feat_file)
        features.append(node_features)

        circles = load_circles(circle_file)
        for circle in circles:
            for node in circle:
                labels.append((int(node), len(circles)))

    features = pd.concat(features, axis=0).reset_index(drop=True)
    return G, features, labels

### **Using the files**

In [ ]:
files = ['./facebook_data/0', './facebook_data/107', './facebook_data/348',
         './facebook_data/414', './facebook_data/686', './facebook_data/698',
         './facebook_data/1684', './facebook_data/1912', './facebook_data/3437',
         './facebook_data/3980']

In [ ]:
G, features, labels = prepare_dataset(files)

### **Adding some extra features**

In [ ]:
def add_graph_features(G, features_df):
    degrees = dict(G.degree())
    clustering_coeffs = nx.clustering(G)

    features_df['degree'] = features_df.index.map(degrees)
    features_df['clustering_coeff'] = features_df.index.map(clustering_coeffs)
    return features_df

features = add_graph_features(G, features)

### **Getting accuracy more than : 95 %**

In [ ]:
# Create training data
node_labels = {node: label for node, label in labels}
X = features.values
y = np.array([node_labels.get(i, -1) for i in range(len(features))])  # Assign -1 for unlabeled nodes

# Filter out unlabeled nodes
X = X[y != -1]
y = y[y != -1]

# --- Apply Label Encoding ---
le = LabelEncoder()
y = le.fit_transform(y)
# --- End of Label Encoding ---

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an XGBoost model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f} %")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:45:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 96.36 %


# **Using Neighbor-based Collaborative Filtering - A basic algorithm**

In [ ]:
def load_facebook_data(folder_path):
    G = nx.Graph()
    features = {}
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith(".edges"):
            with open(file_path, 'r') as f:
                for line in f:
                    node1, node2 = map(int, line.split())
                    G.add_edge(node1, node2)
        elif file.endswith(".feat"):
            with open(file_path, 'r') as f:
                for line in f:
                    data = list(map(int, line.split()))
                    node_id, node_features = data[0], data[1:]
                    features[node_id] = node_features
    return G, features

In [ ]:
def recommend_friends(graph, node, top_k=5):
    scores = defaultdict(int)
    for neighbor in graph.neighbors(node):
        for potential_friend in graph.neighbors(neighbor):
            if potential_friend != node and not graph.has_edge(node, potential_friend):
                scores[potential_friend] += 1
    recommendations = sorted(scores.items(), key=lambda x: -x[1])[:top_k]
    return recommendations

### **Printing Top 10 reccomendation for 3100 node**

In [ ]:
if __name__ == "__main__":
    dataset_folder = "./facebook_data"
    print("Loading dataset...")
    graph, node_features = load_facebook_data(dataset_folder)
    print(f"Graph loaded with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    test_node = 3100
    print(f"Generating friend recommendations for node {test_node}...")
    recommendations = recommend_friends(graph, test_node, top_k=10)
    print("Top recommendations:")
    for friend, score in recommendations:
        print(f"Node {friend} (Score: {score})")

Loading dataset...
Graph loaded with 3959 nodes and 84243 edges.
Generating friend recommendations for node 3100...
Top recommendations:
Node 2775 (Score: 15)
Node 3412 (Score: 14)
Node 3001 (Score: 13)
Node 3051 (Score: 13)
Node 3086 (Score: 13)
Node 2728 (Score: 13)
Node 3113 (Score: 12)
Node 3207 (Score: 12)
Node 2739 (Score: 12)
Node 3076 (Score: 12)


# **Using node2vec embeddings and XGBoost machine learning classifier**

In [ ]:
pip install node2vec

In [ ]:
import os
import pandas as pd
import networkx as nx
import numpy as np
from node2vec import Node2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from collections import defaultdict

### **Load edges and features**

In [ ]:
def load_facebook_data(folder_path):
    G = nx.Graph()
    features = {}
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith(".edges"):
            with open(file_path, 'r') as f:
                for line in f:
                    node1, node2 = map(int, line.split())
                    G.add_edge(node1, node2)
        elif file.endswith(".feat"):
            with open(file_path, 'r') as f:
                for line in f:
                    data = list(map(int, line.split()))
                    node_id, node_features = data[0], data[1:]
                    features[node_id] = node_features
    return G, features

### **Generate node embeddings using node2vec**

In [ ]:
def generate_node_embeddings(G, dimensions=128, walk_length=80, num_walks=10, window=10):
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=4)
    model = node2vec.fit(window=window)
    embeddings = {node: model.wv.get_vector(str(node)) for node in G.nodes()}
    return embeddings

### **Prepare training data for friend recommendation**

In [ ]:
def prepare_training_data(G, embeddings, top_k=10):
    features = []
    labels = []
    for node in G.nodes():
        node_embedding = embeddings[node]
        neighbors = list(G.neighbors(node))
        for neighbor in neighbors:
            neighbor_embedding = embeddings[neighbor]
            features.append(np.concatenate([node_embedding, neighbor_embedding]))
            labels.append(1)
        non_neighbors = [potential_friend for potential_friend in G.nodes() if not G.has_edge(node, potential_friend)]
        np.random.shuffle(non_neighbors)
        for non_friend in non_neighbors[:len(neighbors)]:
            non_friend_embedding = embeddings[non_friend]
            features.append(np.concatenate([node_embedding, non_friend_embedding]))
            labels.append(0)
    return np.array(features), np.array(labels)

### **Train the recommendation model (XGBoost)**

In [ ]:
def train_recommendation_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f} %")
    return model

### **Recommend friends for a given node**

In [ ]:
def recommend_friends(graph, node, embeddings, model, top_k=5):
    node_embedding = embeddings[node]
    scores = []

    for potential_friend in graph.nodes():
        if potential_friend != node and not graph.has_edge(node, potential_friend):
            potential_friend_embedding = embeddings[potential_friend]
            feature_vector = np.concatenate([node_embedding, potential_friend_embedding]).reshape(1, -1)
            score = model.predict_proba(feature_vector)[0][1]
            scores.append((potential_friend, score))
    recommendations = sorted(scores, key=lambda x: -x[1])[:top_k]
    return recommendations

In [ ]:
if __name__ == "__main__":
    # Load data
    dataset_folder = "./facebook_data"
    print("Loading dataset...")
    graph, node_features = load_facebook_data(dataset_folder)
    print(f"Graph loaded with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")

    # Generate node embeddings
    print("Generating node embeddings using node2vec...")
    embeddings = generate_node_embeddings(graph)

    # Prepare training data
    print("Preparing training data...")
    X, y = prepare_training_data(graph, embeddings)

    # Train the recommendation model
    print("Training recommendation model...")
    model = train_recommendation_model(X, y)

    # Generate recommendations for a test node
    test_node = 3100
    print(f"Generating friend recommendations for node {test_node}...")
    recommendations = recommend_friends(graph, test_node, embeddings, model, top_k=10)

    # Display recommendations
    print("Top recommendations:")
    for friend, score in recommendations:
        print(f"Node {friend} (Score: {score})")


Loading dataset...
Graph loaded with 3959 nodes and 84243 edges.
Generating node embeddings using node2vec...


Computing transition probabilities:   0%|          | 0/3959 [00:00<?, ?it/s]

Preparing training data...
Training recommendation model...
Accuracy: 97.68 %
Generating friend recommendations for node 3100...
Top recommendations:
Node 3379 (Score: 0.984081506729126)
Node 2775 (Score: 0.9827460050582886)
Node 2976 (Score: 0.9809519648551941)
Node 2748 (Score: 0.9791770577430725)
Node 3273 (Score: 0.9790619611740112)
Node 2739 (Score: 0.9790334105491638)
Node 2742 (Score: 0.9787536859512329)
Node 2692 (Score: 0.978733241558075)
Node 3177 (Score: 0.9766011238098145)
Node 3106 (Score: 0.9724658131599426)
